# Wykrywanie encji nazwanych z Flair

To już ostatnie laboratoria zadaniowe, w związku z tym, jeśli znajdziecie chwilę wolnego czasu, wypełnijcie proszę ankietę: https://docs.google.com/forms/d/1rHPjpL70XdXRD-ILl3AHophPNUk0AhsFus1-mtkUPsI

Pozwoli to mi poprawić laboratoria w przyszłości, z góry dziękuję :)

# Flair

Biblioteka Flair to bardzo popularne narzędzie do tagowania sekwencji. Zaintstalujmy ją

In [1]:
!pip install flair

You should consider upgrading via the 'c:\users\marcin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 401.9/401.9 KB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 24.0/24.0 MB 1.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 3.6/3.6 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 19.7/19.7 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 788.5/788.5 KB 1.3 MB/s et

In [3]:
# OPCJONALNE WSPARCIE DLA KART GRAFICZNYCH
# W colabie możemy trenować z wykorzystaniem karty graficznej, dzięki temu trening działa dużo szybciej
# Aby włączyć wsparcie dla karty graficznej musimy:
# 1. w menu 'srodowisko wykonawcze' wybrać `zmien typ srodowiska wykonawczego` i tam `akcelerator sprzętowy` = GPU
# 2. odkomentować linijki poniżej
import flair, torch
flair.device = torch.device('cuda:0') 

# Ładowanie zbioru danych i słownika z etykietami.

**Zadanie1: (1 punkt):** Stwórz słownik etykiet z wczytanego korpusu korzystając z funkcji `make_label_dictionary()`. W naszym zbiorze, etykiety do wykrycia występują w kolumnie `ner`, której identyfikator został zapisany w linijce 6. Tutorial: https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md może okazać się pomocny.

Efektem działania powinna być lista etykiet np: 
`Dictionary with 20 tags: <unk>, Variable, Class, Application, User_Interface_Element, Code_Block, Language, Function, Data_Structure, Library, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm`

In [6]:
from flair.datasets import NER_ENGLISH_STACKOVERFLOW    # zbiór otagowanych postów na Stacku

corpus = NER_ENGLISH_STACKOVERFLOW().downsample(0.4)   # pobieramy korpus i zmniejszamy jego wielkość
corpus.filter_empty_sentences()                         # usuwamy puste zdania

label_type = 'ner'   # identyfikator pod którym możemy dostać typy etykiet
label_dict = corpus.make_label_dictionary(label_type)    # TODO
print('\n\nEtykiety do wykrycia')
print(label_dict)

2022-05-28 11:55:55,956 File train has 741 questions and 897 answers.
2022-05-28 11:55:55,979 File test has 249 questions and 315 answers.
2022-05-28 11:55:56,000 File dev has 247 questions and 289 answers.
2022-05-28 11:55:56,001 Reading data from C:\Users\Marcin\.flair\datasets\ner_english_stackoverflow
2022-05-28 11:55:56,002 Train: C:\Users\Marcin\.flair\datasets\ner_english_stackoverflow\train.txt
2022-05-28 11:55:56,003 Dev: C:\Users\Marcin\.flair\datasets\ner_english_stackoverflow\dev.txt
2022-05-28 11:55:56,004 Test: C:\Users\Marcin\.flair\datasets\ner_english_stackoverflow\test.txt
2022-05-28 11:56:01,665 Filtering empty sentences
2022-05-28 11:56:01,683 Corpus: 3705 train + 1174 dev + 1243 test sentences
2022-05-28 11:56:01,684 Computing label dictionary. Progress:


3705it [00:00, 112574.50it/s]

2022-05-28 11:56:01,722 Dictionary created for label 'ner' with 20 values: Application (seen 539 times), Class (seen 535 times), Variable (seen 449 times), User_Interface_Element (seen 371 times), Code_Block (seen 362 times), Language (seen 310 times), Library (seen 298 times), Function (seen 279 times), Data_Structure (seen 233 times), Data_Type (seen 174 times), File_Type (seen 167 times), File_Name (seen 135 times), Version (seen 113 times), HTML_XML_Tag (seen 93 times), Device (seen 76 times), Operating_System (seen 67 times), Website (seen 37 times), User_Name (seen 36 times), Algorithm (seen 18 times)


Etykiety do wykrycia
Dictionary with 20 tags: <unk>, Application, Class, Variable, User_Interface_Element, Code_Block, Language, Library, Function, Data_Structure, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm


# Embeddingi

W narzędziu Flair możemy bardzo prosto składać ze sobą różne embeddingi. 

**Zad2 (2 punkty):** Zapoznaj się z działaniem `StackedEmbeddings` opisanego w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md i zbuduj embeddingi zawierające reprezentacje pochodzące zarówno z Glove jak i Flairowe, oparte na `news-forward`. 

In [7]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings

glove_embedding = WordEmbeddings('glove')

flair_embedding_forward = FlairEmbeddings('news-forward')

embeddings = StackedEmbeddings([glove_embedding, flair_embedding_forward])

In [10]:
from flair.data import Sentence

sentence = Sentence('The grass is green .')

embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token[0]: "The"
tensor([-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -4.4014e-04,
        -3.9301e-02,  1.0601e-02])
Token[1]: "grass"
tensor([-8.1353e-01,  9.4042e-01, -2.4048e-01,  ..., -3.7749e-04,
        -2.3563e-02,  1.1700e-02])
Token[2]: "is"
tensor([-0.5426,  0.4148,  1.0322,  ..., -0.0061,  0.0112,  0.0100])
Token[3]: "green"
tensor([-0.6791,  0.3491, -0.2398,  ..., -0.0026, -0.0118,  0.0455])
Token[4]: "."
tensor([-3.3979e-01,  2.0941e-01,  4.6348e-01,  ..., -2.3405e-04,
         3.8688e-03,  5.7725e-03])


# Tagger i trainer

**Zadanie 3 (2 punkty)** Bazując na treściach opisanych w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md przygotuj obiekt `SequenceTagger`, którego rozmiar wartswy ukrytej wyniesie 256. Do obiektu tego przekażemy stworzone wcześniej embeddingi, słownik `label_dict` i nazwę kolumny z etykietą ze zmiennej `label_type`. Ustawmy `use_crf` na True.

Przygotuj obiekt `ModelTrainer`, który przyjmie zarówno nasz korpus jak i stworzony przed chwilą `SequenceTagger`.

In [9]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)
trainer = ModelTrainer(tagger, corpus)

#stworzony trainer możemy zacząć trenować!
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=5)

# a kiedy wytrenujemy, wczytujemy najlepszy model.
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')


2022-05-28 11:56:19,055 SequenceTagger predicts: Dictionary with 77 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Function, B-Function, E-Function, I-Function, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version
2022-05-28 11:56:19,173 ----------------------------------------------------------------------------------------------------
2022-05-28 11:56:19,174 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbed

c:\users\marcin\appdata\local\programs\python\python38\lib\site-packages\flair\trainers\trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-05-28 11:56:28,276 epoch 1 - iter 11/116 - loss 1.78870488 - samples/sec: 38.78 - lr: 0.100000
2022-05-28 11:56:35,922 epoch 1 - iter 22/116 - loss 1.28483405 - samples/sec: 46.05 - lr: 0.100000
2022-05-28 11:56:43,965 epoch 1 - iter 33/116 - loss 1.08328164 - samples/sec: 43.77 - lr: 0.100000
2022-05-28 11:56:52,153 epoch 1 - iter 44/116 - loss 0.97266513 - samples/sec: 43.01 - lr: 0.100000
2022-05-28 11:57:00,297 epoch 1 - iter 55/116 - loss 0.87712983 - samples/sec: 43.23 - lr: 0.100000
2022-05-28 11:57:08,086 epoch 1 - iter 66/116 - loss 0.82388234 - samples/sec: 45.21 - lr: 0.100000
2022-05-28 11:57:18,208 epoch 1 - iter 77/116 - loss 0.80199892 - samples/sec: 34.79 - lr: 0.100000
2022-05-28 11:57:26,178 epoch 1 - iter 88/116 - loss 0.77496190 - samples/sec: 44.18 - lr: 0.100000
2022-05-28 11:57:34,734 epoch 1 - iter 99/116 - loss 0.74398487 - samples/sec: 41.15 - lr: 0.100000
2022-05-28 11:57:42,557 epoch 1 - iter 110/116 - loss 0.72377483 - samples/sec: 45.01 - lr: 0.100000

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:28<00:00,  1.32it/s]

2022-05-28 11:58:14,903 Evaluating as a multi-label problem: False
2022-05-28 11:58:14,920 DEV : loss 0.4681435823440552 - f1-score (micro avg)  0.0151
2022-05-28 11:58:14,973 BAD EPOCHS (no improvement): 0
2022-05-28 11:58:14,975 saving best model


...

2022-05-28 12:01:43,881 Evaluating as a multi-label problem: False
2022-05-28 12:01:43,901 DEV : loss 0.30565327405929565 - f1-score (micro avg)  0.3172


2022-05-28 12:01:43,956 BAD EPOCHS (no improvement): 0
2022-05-28 12:01:43,957 saving best model
2022-05-28 12:01:54,475 ----------------------------------------------------------------------------------------------------
2022-05-28 12:01:54,477 loading file resources\taggers\example-upos\best-model.pt
2022-05-28 12:01:55,135 SequenceTagger predicts: Dictionary with 79 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Function, B-Function, E-Function, I-Function, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type,

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:24<00:00,  1.60it/s]

2022-05-28 12:02:19,748 Evaluating as a multi-label problem: False
2022-05-28 12:02:19,768 0.465	0.2364	0.3135	0.2137
2022-05-28 12:02:19,770 
Results:
- F-score (micro) 0.3135
- F-score (macro) 0.2488
- Accuracy 0.2137

By class:
                        precision    recall  f1-score   support

                 Class     0.4158    0.4158    0.4158       190
           Application     0.3224    0.2934    0.3072       167
              Variable     0.3333    0.0449    0.0791       156
            Code_Block     0.2564    0.0752    0.1163       133
               Library     0.4808    0.2294    0.3106       109
              Language     0.5366    0.5946    0.5641        74
User_Interface_Element     0.7000    0.1148    0.1972       122
        Data_Structure     0.7755    0.4270    0.5507        89
              Function     0.4444    0.0777    0.1322       103
             File_Name     0.5333    0.2133    0.3048        75
             File_Type     0.5556    0.3261    0.4110        46


2022-05-28 12:02:20,345 SequenceTagger predicts: Dictionary with 79 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-Variable, B-Variable, E-Variable, I-Variable, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Function, B-Function, E-Function, I-Function, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-File_Name, B-File_Name, E-File_Name, I-File_Name, S-Version


# Predykcja z udziałem modelu

Jeśli model został wytrenowany, poniżej znajdziemy fragment kodu, który może wykryć encje w zdaniach.

In [10]:
from flair.data import Sentence

# Jeśli nasz model się wyuczył, powinien wykryć Python jako język.
# Uwaga, ponieważ pracujemy na niewielkim podzbiorze zbioru danych (downsample(0.1) próbkuje 10%), 
# otrzymywane wyniki mogą być kiepskiej jakości, najlepiej zwiększyść ilość danych 
# jeśli pracujemy w domu.
sentence = Sentence('huge files can be opened from Python 3.')   # stwórz obiekt zdania
model.predict(sentence)                                         # wykryj encje nazwane
print(sentence.to_tagged_string())                              # wyświetl zdanie i wykryte w nim encje

Sentence: "huge files can be opened from Python 3 ." → ["Python"/Language]
